In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import time
import mysql.connector
#import MySQLdb as my
from sqlalchemy import create_engine
#from python_mysql_dbconfig import read_db_config

In [40]:
df = pd.read_csv("Privacy_Rights_Clearinghouse-Data-Breaches-Export.csv")

In [41]:
df.head()

,Date Made Public,Company,Type of breach,Type of organization,Total Records,Description of incident,Information Source,Source URL
0,7-Oct-15,LoopPay,HACK,BSO,0,"""Months before its technology became the cente...",Media,https://www.nytimes.com/2015/10/08/technology/...
1,1-Sep-16,New York State Psychiatric Institute,HACK,GOV,"21,880","""The New York State Office of Mental Health (O...",Media,https://healthitsecurity.com/news/ny-psychiatr...
2,8-Jan-17,E-Sports Entertainment Association (ESEA),HACK,BSO,"1,503,710","""E-Sports Entertainment Association (ESEA), on...",Media,http://www.csoonline.com/article/3155397/secur...
3,10-Jan-17,Legal Aid Society of Orange County (LASOC),DISC,NGO,0,"""LASOC developed the I-CAN! web application, w...",California Attorney General,NaN
4,3-Jan-17,MetroPlus Health Plan,DISC,MED,808,As reported by Health and Human Services unaut...,Government Agency,NaN


infomration_public_date, source_url, incident_summary, reported_by will be information source --> incident_report
breach_type --> attack
victim_type, victim_name --> victim table
Location --> Demographics

In [42]:
incident_report_df = df[['Date Made Public', 'Source URL', 'Description of incident', 'Information Source']]
victim_df = df[['Type of organization', 'Company']]

In [43]:
incident_report_df.columns = ['reported_date','source_url', 'incident_summary', 'reported_by']
victim_df.columns = ['victim_type', 'victim_name']

In [16]:
for row in incident_report_df:
    print (type(incident_report_df[row][1]))

<class 'str'>
<class 'str'>
<class 'str'>


In [44]:
incident_report_df.head()

,reported_date,source_url,incident_summary,reported_by
0,7-Oct-15,https://www.nytimes.com/2015/10/08/technology/...,"""Months before its technology became the cente...",Media
1,1-Sep-16,https://healthitsecurity.com/news/ny-psychiatr...,"""The New York State Office of Mental Health (O...",Media
2,8-Jan-17,http://www.csoonline.com/article/3155397/secur...,"""E-Sports Entertainment Association (ESEA), on...",Media
3,10-Jan-17,NaN,"""LASOC developed the I-CAN! web application, w...",California Attorney General
4,3-Jan-17,NaN,As reported by Health and Human Services unaut...,Government Agency


In [50]:
#Converting string to datetime format

for index, row in incident_report_df.iterrows():
    datestring = row['reported_date']
    row['reported_date'] = time.strptime(datestring,"%d-%b-%y")
    
for row in incident_report_df:
    print (type(incident_report_df[row][1]))

<class 'time.struct_time'>
<class 'str'>
<class 'str'>
<class 'str'>


In [54]:
#Replacing all the null values with Missing Values string
incident_report_df = incident_report_df.replace(np.nan, 'Missing Values', regex=True)

In [17]:
#for index, row in incident_report_df.iterrows():
    #print (row['source_url'])

In [55]:
#Ingesting incident_report_df to incident report table
engine = mysql.connector.connect(user='root', password='xx', database='wikibreach')
cursor = engine.cursor()

for index, rows in incident_report_df.iterrows():
    insert_stmt = "INSERT INTO incident_report (reported_date, source_url, incident_summary, reported_by) VALUES (%s, %s, %s, %s)"
    data = (rows.reported_date, rows.source_url, rows.incident_summary, rows.reported_by)
    cursor.execute(insert_stmt, data)
    
    engine.commit()

cursor.close()
engine.close()


In [57]:
victim_df.head()

,victim_type,victim_name
0,BSO,LoopPay
1,GOV,New York State Psychiatric Institute
2,BSO,E-Sports Entertainment Association (ESEA)
3,NGO,Legal Aid Society of Orange County (LASOC)
4,MED,MetroPlus Health Plan


In [59]:
#Ingesting victim_df to victim table
engine = mysql.connector.connect(user='root', password='xx', database='wikibreach')
cursor = engine.cursor()

for index, rows in victim_df.iterrows():
    insert_stmt = "INSERT INTO victim (victim_type, victim_name) VALUES (%s, %s)"
    data = (rows.victim_type, rows.victim_name)
    cursor.execute(insert_stmt, data)
    
    engine.commit()

cursor.close()
engine.close()